In [1]:
import pandas as pd
import numpy as np
import os

In [19]:
train = pd.read_csv('/Users/krybkin/Desktop/ods_hackaton/train_data.csv')
train_complaints = pd.read_csv('/Users/krybkin/Desktop/ods_hackaton/complaints_train.csv')
test = pd.read_csv('/Users/krybkin/Desktop/ods_hackaton/test_data.csv')
sub = pd.read_csv('/Users/krybkin/Desktop/ods_hackaton/sample_submission.csv')

### Filtering columns

In [99]:
train_agg = train.groupby('unique_id', as_index=False).last()
test_agg = test.groupby('unique_id', as_index=False).last()

In [206]:
needed_columns = ['Минимальная глубина CPV (2017 год)', 'СуммаТендера', 'Организатор',
                 'Дата начала подачи предложений', 'Тип методу закупівлі',
                 'Тип организатора', 'Статус лота', 'СтатусТендера',
                 'РегионОрганизатора', 'Регион поставки', 'Тендер',
                 'Дата начала периода уточнений', 'Дата завершения',
                 'Дата завершения подачи предложений', 'ШагУменьшенияЛот','СуммаГарантииПредложенияЛота']

In [ ]:
train_agg = train_agg[needed_columns]
test_agg = test_agg[needed_columns]

### Replacing index

In [189]:
train_agg.index = train_agg.unique_id
train_agg.drop('unique_id', axis=1, inplace=True)
test.index = test.unique_id
test.drop('unique_id', axis=1, inplace=True)

In [107]:
def filter_digits(row):
    if row == '-':
        return '0000'
    else:
        return row[:4]

In [159]:
train_agg['Минимальная глубина CPV (2017 год)'] = train_agg['Минимальная глубина CPV (2017 год)'].apply(lambda x: filter_digits(x))
test_agg['Минимальная глубина CPV (2017 год)'] = test_agg['Минимальная глубина CPV (2017 год)'].apply(lambda x: filter_digits(x))

In [136]:
merged_train = train_agg.join(train.groupby('unique_id').count()['ID'].to_frame())

In [147]:
merged_train['Дата завершения подачи предложений'] = pd.to_datetime(merged_train['Дата завершения подачи предложений'])
merged_train['Дата завершения'] = pd.to_datetime(merged_train['Дата завершения'])

In [172]:
merged_train['target'] = merged_train['ID']
merged_train.drop('ID', axis=1, inplace=True)

### Saving datasets

In [ ]:
merged_train.to_csv('/Users/krybkin/Desktop/ods_hackaton/train_agg_2.csv')

In [192]:
test.to_csv('/Users/krybkin/Desktop/ods_hackaton/test.csv')